![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

## Entraînement du modèle dans un Notebook (QuantBook)

In [1]:
###################################################
# NOTEBOOK QUANTCONNECT - ML EXAMPLE (One Cell)
# Ajout d'indicateurs : multiple EMAs, ADX, ATR, etc.
###################################################

# =========================
#         PARAMETRES
# =========================
MODEL_KEY         = "myCryptoMlModel.pkl"  # Clé pour l'Object Store
TRAIN_START_DATE  = datetime(2022, 1, 1)
TRAIN_END_DATE    = datetime(2023, 1, 1)
MODEL_CHOICE      = "RF"  # "RF" (RandomForest), "SVC", ou "XGB"
USE_SPLIT_RATIO   = 0.8   # 80% des données pour train, 20% pour test

# Ici, on peut commenter/décommenter selon les indicateurs qu'on veut tester
FEATURES_SELECTED = [
    "SMA20",      # SMA 20
    "RSI",        # RSI 14
    "DailyReturn",
    "EMA_10",
    "EMA_20",
    "EMA_50",
    "EMA_200",
    "ADX_14",
    "ATR_14"
]

# =========================
#       IMPORTS
# =========================
from AlgorithmImports import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ta.trend import SMAIndicator, EMAIndicator, ADXIndicator
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Pour les modèles
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

# =========================
#   QUANTBOOK INIT
# =========================
qb = QuantBook()

# =========================
#   CHOIX DU MODELE
# =========================
def get_model(model_choice):
    if model_choice.upper() == "RF":
        return RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    elif model_choice.upper() == "SVC":
        return SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)
    elif model_choice.upper() == "XGB":
        return xgboost.XGBClassifier(
            n_estimators=100,
            max_depth=5,
            random_state=42,
            use_label_encoder=False,
            eval_metric="logloss"
        )
    else:
        raise ValueError(f"Modèle inconnu: {model_choice}")

model = get_model(MODEL_CHOICE)

# =========================
#   CHARGEMENT DES DONNEES
# =========================
symbol = qb.AddCrypto("BTCUSDT", Resolution.Daily, Market.Binance).Symbol
history = qb.History(symbol, TRAIN_START_DATE, TRAIN_END_DATE, Resolution.Daily)
print(f"History bars: {len(history)}")

df = (history.reset_index()
      .pivot(index='time', columns='symbol', values='close')
      .rename(columns={symbol:'Close'}))
df.sort_index(inplace=True)

# =========================
#   CALCUL DES INDICATEURS
# =========================
# Déjà existants :
df["SMA20"]       = SMAIndicator(df["Close"], 20, fillna=True).sma_indicator()
df["RSI"]         = RSIIndicator(df["Close"], 14, fillna=True).rsi()
df["DailyReturn"] = df["Close"].pct_change()

# Nouveaux EMAs
df["EMA_10"]  = EMAIndicator(df["Close"], window=10, fillna=True).ema_indicator()
df["EMA_20"]  = EMAIndicator(df["Close"], window=20, fillna=True).ema_indicator()
df["EMA_50"]  = EMAIndicator(df["Close"], window=50, fillna=True).ema_indicator()
df["EMA_200"] = EMAIndicator(df["Close"], window=200, fillna=True).ema_indicator()

# ADX (sur 14 jours)
# Besoin du high, low, close => on va les générer
# Mais dans le backtest "history" on n'a que 'close' par défaut,
#   donc l'ADX sera fictif si on n'a pas le high/low. 
#   => Sur QC live/backtest, possible d'utiliser .High, .Low. 
#   => Ici, on simule via un hack: 
df["High"] = df["Close"] * 1.01  # hypothetique +1%
df["Low"]  = df["Close"] * 0.99  # hypothetique -1%

adx = ADXIndicator(high=df["High"], low=df["Low"], close=df["Close"], window=14, fillna=True)
df["ADX_14"] = adx.adx()

# ATR (14)
atr = AverageTrueRange(high=df["High"], low=df["Low"], close=df["Close"], window=14, fillna=True)
df["ATR_14"] = atr.average_true_range()

df.dropna(inplace=True)

# =========================
#   CREATION DE LA TARGET
# =========================
df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
df.dropna(inplace=True)

# =========================
#   BUILD X, Y
# =========================
X = df[FEATURES_SELECTED].values
y = df["Target"].values

split_idx = int(len(X)*USE_SPLIT_RATIO)
X_train, y_train = X[:split_idx], y[:split_idx]
X_test,  y_test  = X[split_idx:], y[split_idx:]

# =========================
#   ENTRAINEMENT
# =========================
model.fit(X_train, y_train)

# =========================
#   EVALUATION
# =========================
y_pred = model.predict(X_test)
acc    = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")
print(classification_report(y_test, y_pred))

# =========================
#   SAUVEGARDE DANS OBJECT STORE
# =========================
path = qb.ObjectStore.GetFilePath(MODEL_KEY)
joblib.dump(model, path)
qb.ObjectStore.Save(MODEL_KEY)
print(f"Modèle sauvegardé en ObjectStore : {MODEL_KEY}")
